In [1]:
#hide
#default_exp actions
from nbdev.showdoc import show_doc
from nbdev.export import notebook2script
import shutil

# Actions

> Functions and scripts that act on a KiCad project

* toc: true

In [2]:
#export
import os
import sys
import pkg_resources
import subprocess

import jinja2
from fastcore.script import *
import pandas as pd

from kicad_helpers import *
from kicad_helpers.utilities import _set_root

In [3]:
#export
@call_parse
def update_templates(v:Param("verbose", bool),
                     overwrite:Param("overwrite existing templates", bool),
                     root:Param("project root directory", str)="."):
    """
    Install various templates from the `kicad_helpers/templates` directory
    (ignoring anything in the project's `.gitignore` list).
    """
    templates_path = os.path.abspath(pkg_resources.resource_filename('kicad_helpers', 'templates'))
    root = _set_root(root)
    metadata = get_project_metadata(root)
    file_list = []
    exists_flag = False
    for root_, dirs, files in os.walk(templates_path):
        if len(files):
            for file in files:
                path = os.path.join(root_[len(templates_path) + 1:], file)
                if not in_gitignore(path):
                    src_path = os.path.abspath(os.path.join(templates_path, path))
                    dst_path = os.path.abspath(os.path.join(root, path))
                    
                    # Create the `dst_path` directory if it doesn't exist
                    os.makedirs(os.path.split(dst_path)[0], exist_ok=True)

                    if os.path.exists(dst_path):
                        if not overwrite:
                            if v:
                                print(f"{ path } already exists")
                                exists_flag = True
                            continue
    
                    with open(src_path) as f:
                        template = jinja2.Template(f.read())

                    with open(dst_path, "w") as f:
                        if v:
                            print(f"Rendering { path } template.")
                        f.write(template.render(**metadata))
                        
    if not overwrite and exists_flag:
        print("To overwrite existing files, use the --overwrite flag.")

Similar to [nbdev_new](https://nbdev.fast.ai/tutorial.html#Set-up-Repo). Templates are stored in the [kicad_helpers/templates](https://github.com/ryanfobel/kicad-helpers/tree/main/kicad_helpers/templates) folder and are included with the python package by adding `graft kicad_helpers/templates` to the [MANIFEST.in](https://github.com/ryanfobel/kicad-helpers/blob/main/MANIFEST.in) file.

In [4]:
#hide

# Install the patched version of kifield if it isn't already installed
# See: https://github.com/devbisme/KiField/issues/59#issuecomment-952298997
if subprocess.check_output([sys.executable, "-m", "kifield", "--help"]).decode("utf-8").find('addquantidty') > 0:
    install_python_package("git+https://github.com/ryanfobel/KiField.git@add-quantity")
root = os.path.join(get_git_root("."), "_temp")
setup_test_repo(root)

```sh
> kh_update --help
```

In [5]:
#hide_input
print(subprocess.check_output(f"kh_update --help", shell=True).decode("utf-8"))

usage: kh_update [-h] [--v] [--overwrite] [--root ROOT]

Install various templates from the `kicad_helpers/templates` directory (ignoring anything in the project's `.gitignore`
list).

optional arguments:
  -h, --help   show this help message and exit
  --v          verbose (default: False)
  --overwrite  overwrite existing templates (default: False)
  --root ROOT  project root directory (default: .)



```sh
> kh_update --v --overwrite
```

In [6]:
#hide_input
print(subprocess.check_output(f"kh_update --v --overwrite --root { root }", shell=True).decode("utf-8"))

Rendering kitspace.yaml template.
Rendering settings.ini template.
Rendering .github/workflows/build.yml template.
Rendering .kicad_helpers_config/pcb_pdf.yaml template.
Rendering .kicad_helpers_config/pcb_svg.yaml template.
Rendering .kicad_helpers_config/sch_pdf.yaml template.
Rendering .kicad_helpers_config/sch_svg.yaml template.
Rendering .kicad_helpers_config/manufacturers/default.yaml template.
Rendering .kicad_helpers_config/manufacturers/PCBWay.yaml template.
Rendering tests/tests.ipynb template.



In [7]:
#export
@call_parse
def sch_to_bom(root:Param("project root directory", str)=".",
               v:Param("verbose", bool)=False,
               overwrite:Param("update existing schematic", bool)=False):
    """
    Update/create BOM from KiCad schematic.
    """
    root = _set_root(root)
    cmd = f"{ sys.executable } -m kifield --nobackup --overwrite --group -aq -x { get_schematic_path(root) } -i { get_bom_path(root) }"
    if v:
        print(cmd)
    subprocess.check_output(cmd, shell=True)

This function can also be called via a command line script:

```sh
> kh_sch_to_bom --help
```

In [8]:
#hide_input
print(subprocess.check_output("kh_sch_to_bom --help", shell=True).decode("utf-8"))

usage: kh_sch_to_bom [-h] [--root ROOT] [--v] [--overwrite]

Update/create BOM from KiCad schematic.

optional arguments:
  -h, --help   show this help message and exit
  --root ROOT  project root directory (default: .)
  --v          verbose (default: False)
  --overwrite  update existing schematic (default: False)



Running `kh_sch_to_bom` within your project dictory will generate a BOM file (e.g., `manufacturing/default/project-name-BOM.csv`) by extracting fields from the KiCad schematic file (e.g., `project-name.sch`).

```sh
> kh_sch_to_bom --v
```

In [9]:
#hide_input
print(subprocess.check_output(f"kh_sch_to_bom --v --root { root }", shell=True).decode("utf-8"))

/home/ryan/miniconda3/envs/kh/bin/python3.9 -m kifield --nobackup --overwrite --group -aq -x /mnt/c/Users/ryan/OneDrive/dev/python/kicad-helpers/_temp/40-channel-hv-switching-board.sch -i /mnt/c/Users/ryan/OneDrive/dev/python/kicad-helpers/_temp/manufacturing/default/40-channel-hv-switching-board-BOM.csv



Here's an example of a BOM extracted from a KiCad schematic.

In [10]:
#hide_input
df = pd.read_csv(get_bom_path(root))
df

,Refs,Quantity,MPN,Manufacturer,datasheet,footprint,value
0,"C1, C3, C5, C6, C8-C13, C16",11,CL21B104KBCNNNC,Samsung,NaN,Capacitors_SMD:C_0805,0.1uF
1,"C14, C15",2,CL21C151JBANNNC,Samsung,NaN,Capacitors_SMD:C_0805,150pF
2,"C2, C4",2,T491D336K020AT,KEMET,NaN,Sci-Bots:SM2917,33uF
3,C7,1,CL21B103KCANNNC,Samsung,NaN,Capacitors_SMD:C_0805,0.01uF
4,DS1,1,150080BS75000,Würth Elektronik,NaN,LEDs:LED_0805,+3.3V_PWR
5,FB1-FB5,5,742792040,Würth Elektronik,NaN,Resistors_SMD:R_0805,FERRITE
6,JP1,1,NaN,NaN,NaN,Resistors_SMD:R_0805,JUMPER
7,P1,1,NaN,NaN,NaN,Pin_Headers:Pin_Header_Angled_1x06,CONN_01X06
8,P2,1,NaN,NaN,NaN,Pin_Headers:Pin_Header_Straight_2x03,CONN_01X06
9,P3-P6,4,NaN,NaN,NaN,Connect:1pin,CONN_01X01


In [11]:
#export
@call_parse
def bom_to_sch(root:Param("project root directory", str)=".",
               v:Param("verbose", bool)=False,
               overwrite:Param("update existing schematic", bool)=False):
    """
    Update KiCad schematic from BOM file.
    """
    root = _set_root(root)
    cmd = f"{ sys.executable } -m kifield --nobackup --overwrite --fields ~quantity -x { get_bom_path(root) } -i { get_schematic_path(root) }"
    if v:
        print(cmd)
    subprocess.check_output(cmd, shell=True)

This function can also be called via a command line script:

```sh
> kh_bom_to_sch --help
```

In [12]:
#hide_input
print(subprocess.check_output("kh_bom_to_sch --help", shell=True).decode("utf-8"))

usage: kh_bom_to_sch [-h] [--root ROOT] [--v] [--overwrite]

Update KiCad schematic from BOM file.

optional arguments:
  -h, --help   show this help message and exit
  --root ROOT  project root directory (default: .)
  --v          verbose (default: False)
  --overwrite  update existing schematic (default: False)



Running `kh_bom_to_sch` within your project dictory will read the BOM file (e.g., `manufacturing/default/project-name-BOM.csv`), and import the fields back into the KiCad schematic file (e.g., `project-name.sch`).

```sh
> kh_sch_to_bom --v
```

In [13]:
#hide_input
print(subprocess.check_output(f"kh_bom_to_sch --v --root { root }", shell=True).decode("utf-8"))

/home/ryan/miniconda3/envs/kh/bin/python3.9 -m kifield --nobackup --overwrite --fields ~quantity -x /mnt/c/Users/ryan/OneDrive/dev/python/kicad-helpers/_temp/manufacturing/default/40-channel-hv-switching-board-BOM.csv -i /mnt/c/Users/ryan/OneDrive/dev/python/kicad-helpers/_temp/40-channel-hv-switching-board.sch



In [14]:
#export
def run_kibot_docker(config:Param(f"kibot configuation file", str),
              root:Param("project root directory", str)=".",
              v:Param("verbose", bool)=False,
              output:Param("output path relative to ROOT")="."):
    """
    Run kibot script in a local docker container.
    """
    root = _set_root(root)
    if os.path.abspath(output) == output:
        print(f"OUTPUT cannot be an absolute path; it must be relative to ROOT={ root }.")
        return 1
    
    UID = subprocess.check_output("id -u", shell=True).decode("utf-8").strip()
    GID = subprocess.check_output("id -g", shell=True).decode("utf-8").strip()    
   
    cmd = (f"docker run --rm -v { os.path.abspath(root) }:/workdir --workdir=\"/workdir\" "
           f"setsoft/kicad_auto_test:latest "
           f"/bin/bash -c \"useradd --shell /bin/bash -u { UID } -o -c '' -m docker && "
           f"runuser docker -c 'kibot -c { config } "
           f"-e { get_schematic_path(root)[len(root) + 1:] } "
           f"-b { get_board_path(root)[len(root) + 1:] } "
           f"-d { output }'\""
    )
    if v:
        print(cmd)
    subprocess.check_output(cmd, shell=True)

In [15]:
#export
@call_parse
def export_manufacturing(root:Param("project root directory", str)=".",
                         manufacturer:Param(f"\"default\" or manufacturer name", str)="default",
                         v:Param("verbose", bool)=False,
                         output:Param("output path relative to ROOT")="."):
    """
    Export manufacturing files (gerber, drill, and position) by running kibot in a local docker container.
    """
    root = _set_root(root)
    if manufacturer not in get_manufacturers(root):
        print(f"MANUFACTURER must be one of the following: { ', '.join(get_manufacturers(root)) }.")
        return 1
    
    config = f".kicad_helpers_config/manufacturers/{ manufacturer }.yaml"
    run_kibot_docker(config=config, root=root, v=v, output=output)

This function can also be called via a command line script:

> kh_export_man --help

In [16]:
#hide_input
print(subprocess.check_output("kh_export_man --help", shell=True).decode("utf-8"))

usage: kh_export_man [-h] [--root ROOT] [--manufacturer MANUFACTURER] [--v] [--output OUTPUT]

Export manufacturing files (gerber, drill, and position) by running kibot in a local docker container.

optional arguments:
  -h, --help                   show this help message and exit
  --root ROOT                  project root directory (default: .)
  --manufacturer MANUFACTURER  "default" or manufacturer name (default: default)
  --v                          verbose (default: False)
  --output OUTPUT              output path relative to ROOT (default: .)



Running `kh_export_man` within your project directory will export manufacturing outputs (e.g., gerber, drill, and position files).

```sh
> kh_export_man --output outputs
> tree outputs
```

In [17]:
#hide_input
output_path = os.path.join(root, "outputs")

def remove_test_outputs():
    """Remove test outputs if the directory already exists."""
    if os.path.exists(output_path):
        shutil.rmtree(output_path)

print(subprocess.check_output(f"kh_export_man --root { root } --output { output_path[len(root) + 1:] }", shell=True).decode("utf-8"))
outputs = subprocess.check_output(f"cd { os.path.join(root, output_path[len(root) + 1:]) } && tree .", shell=True).decode("utf-8")
assert outputs == '.\n├── gerbers\n│\xa0\xa0 ├── 40-channel-hv-switching-board-NPTH.drl\n│\xa0\xa0 ├── 40-channel-hv-switching-board.drl\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gbl\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gbo\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gbp\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gbs\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gl2\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gl3\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gm1\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gtl\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gto\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gtp\n│\xa0\xa0 └── 40-channel-hv-switching-board.gts\n└── position\n    ├── bottom_pos.pos\n    └── top_pos.pos\n\n2 directories, 15 files\n'
print(outputs)
remove_test_outputs()


.
├── gerbers
│   ├── 40-channel-hv-switching-board-NPTH.drl
│   ├── 40-channel-hv-switching-board.drl
│   ├── 40-channel-hv-switching-board.gbl
│   ├── 40-channel-hv-switching-board.gbo
│   ├── 40-channel-hv-switching-board.gbp
│   ├── 40-channel-hv-switching-board.gbs
│   ├── 40-channel-hv-switching-board.gl2
│   ├── 40-channel-hv-switching-board.gl3
│   ├── 40-channel-hv-switching-board.gm1
│   ├── 40-channel-hv-switching-board.gtl
│   ├── 40-channel-hv-switching-board.gto
│   ├── 40-channel-hv-switching-board.gtp
│   └── 40-channel-hv-switching-board.gts
└── position
    ├── bottom_pos.pos
    └── top_pos.pos

2 directories, 15 files



You can generate manufacturer-specific outputs using the `--manufacturer` flag, e.g.:

```sh
> kh_export_man --manufacturer PCBWay --output outputs
```

Support for additional manufacturers can be added by creating a `*.yaml` file in the [.kicad_helpers_config/manufacturers](https://github.com/ryanfobel/kicad-helpers/tree/main/kicad_helpers/templates/.kicad_helpers_config/manufacturers) directory.

In [18]:
#hide

# Test PCBWay manufacturing outputs
subprocess.check_output(f"kh_export_man --v --manufacturer PCBWay --root { root } --output { output_path[len(root) + 1:] }", shell=True).decode("utf-8")
outputs = subprocess.check_output(f"cd { os.path.join(root, output_path[len(root) + 1:]) } && tree -I $(python -c \"from kicad_helpers.utilities import get_gitignore_list; print('kicad_helpers|' + get_gitignore_list())\") .", shell=True).decode("utf-8")
assert outputs == outputs
remove_test_outputs()

# Test a manufacturer that has no *.yaml configuration file
output = subprocess.check_output(f"kh_export_man --root { root } --manufacturer bad --output { output_path[len(root) + 1:] }", shell=True).decode("utf-8")
assert output == 'MANUFACTURER must be one of the following: default, PCBWay.\n'

# Test using an absolute path for OUTPUT
output = subprocess.check_output(f"kh_export_man --root { root } --output { output_path }", shell=True).decode("utf-8")
assert(output.startswith("OUTPUT cannot be an absolute path; it must be relative to ROOT="))

In [19]:
#export
@call_parse
def export_sch(root:Param("project root directory", str)=".",
               ext:Param(f"svg or pdf", str)="pdf",
               v:Param("verbose", bool)=False,
               output:Param("output path relative to ROOT")="."):
    """
    Export the schematic by running kibot in a local docker container.
    """
    root = _set_root(root)
    supported_types = ["svg", "pdf"]
    
    if ext not in supported_types:
        print(f"EXT must be one of: { ','.join(supported_types) }.")
        return 1

    config = f".kicad_helpers_config/sch_{ ext }.yaml"
    run_kibot_docker(config=config, root=root, v=v, output=output)

This function can also be called via a command line script:

> kh_export_sch --help

In [20]:
#hide_input
print(subprocess.check_output("kh_export_sch --help", shell=True).decode("utf-8"))

usage: kh_export_sch [-h] [--root ROOT] [--ext EXT] [--v] [--output OUTPUT]

Export the schematic by running kibot in a local docker container.

optional arguments:
  -h, --help       show this help message and exit
  --root ROOT      project root directory (default: .)
  --ext EXT        svg or pdf (default: pdf)
  --v              verbose (default: False)
  --output OUTPUT  output path relative to ROOT (default: .)



In [21]:
#hide_input
print(subprocess.check_output(f"kh_export_sch --root { root } --ext pdf --output { output_path[len(root) + 1:] }", shell=True).decode("utf-8"))
outputs = subprocess.check_output(f"cd { os.path.join(root, output_path[len(root) + 1:]) } && tree .", shell=True).decode("utf-8")
assert outputs == '.\n└── 40-channel-hv-switching-board-schematic.pdf\n\n0 directories, 1 file\n'
print(outputs)
remove_test_outputs()


.
└── 40-channel-hv-switching-board-schematic.pdf

0 directories, 1 file



In [22]:
#hide_input
print(subprocess.check_output(f"kh_export_sch --root { root } --ext svg --output { output_path[len(root) + 1:] }", shell=True).decode("utf-8"))
outputs = subprocess.check_output(f"cd { os.path.join(root, output_path[len(root) + 1:]) } && tree .", shell=True).decode("utf-8")
assert outputs == '.\n├── 40-channel-hv-switching-board-schematic.svg\n├── switches_0-19-switches_0-19.svg\n└── switches_20-39-switches_20-39.svg\n\n0 directories, 3 files\n'
print(outputs)
remove_test_outputs()


.
├── 40-channel-hv-switching-board-schematic.svg
├── switches_0-19-switches_0-19.svg
└── switches_20-39-switches_20-39.svg

0 directories, 3 files



In [23]:
#export
@call_parse
def export_pcb(root:Param("project root directory", str)=".",
               ext:Param(f"svg or pdf", str)="pdf",
               v:Param("verbose", bool)=False,
               output:Param("output path relative to ROOT")="."):
    """
    Export the pcb layout by running kibot in a local docker container.
    """
    root = _set_root(root)
    supported_types = ["svg", "pdf"]
    
    if ext not in supported_types:
        print(f"EXT must be one of: { ','.join(supported_types) }.")
        return 1
    
    config = f".kicad_helpers_config/pcb_{ ext }.yaml"
    run_kibot_docker(config=config, root=root, v=v, output=output)

This function can also be called via a command line script:

> kh_export_pcb --help

In [24]:
#hide_input
print(subprocess.check_output("kh_export_pcb --help", shell=True).decode("utf-8"))

usage: kh_export_pcb [-h] [--root ROOT] [--ext EXT] [--v] [--output OUTPUT]

Export the pcb layout by running kibot in a local docker container.

optional arguments:
  -h, --help       show this help message and exit
  --root ROOT      project root directory (default: .)
  --ext EXT        svg or pdf (default: pdf)
  --v              verbose (default: False)
  --output OUTPUT  output path relative to ROOT (default: .)



In [25]:
#hide_input
print(subprocess.check_output(f"kh_export_pcb --root { root } --ext pdf --output { output_path[len(root) + 1:] }", shell=True).decode("utf-8"))
outputs = subprocess.check_output(f"cd { os.path.join(root, output_path[len(root) + 1:]) } && tree .", shell=True).decode("utf-8")
#assert outputs == '.\n└── 40-channel-hv-switching-board-schematic.pdf\n\n0 directories, 1 file\n'
print(outputs)
remove_test_outputs()


.
├── 40-channel-hv-switching-board-3_3V.pdf
├── 40-channel-hv-switching-board-B_Mask.pdf
├── 40-channel-hv-switching-board-B_Paste.pdf
├── 40-channel-hv-switching-board-B_SilkS.pdf
├── 40-channel-hv-switching-board-Back.pdf
├── 40-channel-hv-switching-board-Edge_Cuts.pdf
├── 40-channel-hv-switching-board-F_Mask.pdf
├── 40-channel-hv-switching-board-F_Paste.pdf
├── 40-channel-hv-switching-board-F_SilkS.pdf
├── 40-channel-hv-switching-board-Front.pdf
└── 40-channel-hv-switching-board-GND.pdf

0 directories, 11 files



In [26]:
#hide_input
print(subprocess.check_output(f"kh_export_pcb --root { root } --ext svg --output { output_path[len(root) + 1:] }", shell=True).decode("utf-8"))
outputs = subprocess.check_output(f"cd { os.path.join(root, output_path[len(root) + 1:]) } && tree .", shell=True).decode("utf-8")
#assert outputs == '.\n└── 40-channel-hv-switching-board-schematic.pdf\n\n0 directories, 1 file\n'
print(outputs)
remove_test_outputs()


.
├── 40-channel-hv-switching-board-3_3V.svg
├── 40-channel-hv-switching-board-B_Mask.svg
├── 40-channel-hv-switching-board-B_Paste.svg
├── 40-channel-hv-switching-board-B_SilkS.svg
├── 40-channel-hv-switching-board-Back.svg
├── 40-channel-hv-switching-board-Edge_Cuts.svg
├── 40-channel-hv-switching-board-F_Mask.svg
├── 40-channel-hv-switching-board-F_Paste.svg
├── 40-channel-hv-switching-board-F_SilkS.svg
├── 40-channel-hv-switching-board-Front.svg
└── 40-channel-hv-switching-board-GND.svg

0 directories, 11 files



In [27]:
#hide
notebook2script()

Converted 00_actions.ipynb.
Converted 01_test.ipynb.
Converted 02_utilities.ipynb.
Converted index.ipynb.
